# Inspecting the SRRL weather data and the pyranometer data
The SRRL data and the pyranometer data will be plotted to determine the sunniest period for BR modeling of the irradiance distribution at the BARN facility at NREL Golden campus.

In [1]:
# First, load the packages
import os
from pathlib import Path
import numpy as np
import pandas as pd
import bifacial_radiance as br

# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)

Working on a  Windows 10
Python version  3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.0.3


## Data wrangling
### SRRL data
1. Create a datetime variable and set it as the index
2. Change the data to a tidy form (until I can figure out how to work with plotly in a long form)

### Measured pyranometer data
1. Create a datetime variable and set it as the index
2. Change the data to a tidy form

In [42]:
# Load the measured data
measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1) # The field pyranometer data
srrl_weather = pd.read_csv(str(Path().resolve() / 'WeatherFiles' /  'PSM3_15T.csv'), header = 2) # Data from SRRL

## SRRL data
# Create a Datetime variable and set it as the index
srrl_weather['date'] = srrl_weather[['Year','Month','Day']].astype(str).agg('-'.join, axis=1) # Couldn't think of a proper way to chain this process into a single command. If you think of something, please let me know
srrl_weather['time'] = srrl_weather[['Hour','Minute']].astype(str).agg(':'.join, axis=1)
srrl_weather['Datetime'] = pd.to_datetime(srrl_weather[['date','time']].agg(" ".join, axis=1))
srrl_weather.set_index('Datetime', inplace=True)

## Measured pyranometer data
# Create a Datetime variable for the measured data using the existing 'TIMESTAMP' column.
measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
measured = measured.drop(["TIMESTAMP", "RECORD"], axis = 'columns') # drop this column so the resample command does not confuse TIMESTAMP and Datetime columns

measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='left', closed='right').mean() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.

## Join two datasets
srrlandmeasured = pd.concat([srrl_weather, measured], axis = 1, join = 'inner')


In [43]:
# print(measured)
# print(srrl_weather)
print(srrlandmeasured)

                     Year  Month  Day  Hour  Minute      Wspd       Tdry  \
Datetime                                                                   
2023-09-07 11:15:00  2023      9    7    11      15  2.345533  28.330667   
2023-09-07 11:30:00  2023      9    7    11      30  2.286067  28.506667   
2023-09-07 11:45:00  2023      9    7    11      45  1.852600  29.238667   
2023-09-07 12:00:00  2023      9    7    12       0  1.430867  29.086000   
2023-09-07 12:15:00  2023      9    7    12      15  1.797600  29.950000   
...                   ...    ...  ...   ...     ...       ...        ...   
2023-11-01 22:45:00  2023     11    1    22      45  1.170867  10.070667   
2023-11-01 23:00:00  2023     11    1    23       0  1.233467  10.319333   
2023-11-01 23:15:00  2023     11    1    23      15  0.489400  10.398000   
2023-11-01 23:30:00  2023     11    1    23      30  0.640267   9.451333   
2023-11-01 23:45:00  2023     11    1    23      45  1.553600   9.658000   

           

In [13]:
import plotly.express as px
import plotly.graph_objects as go

        nation  gold  silver  bronze
0  South Korea    24      13      11
1        China    10      15       8
2       Canada     9      12      12


In [48]:
fig1 = px.line(srrlandmeasured.reset_index(), x = 'Datetime', y = ['GHI', 'DNI'])
fig1.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Irradiance type"
    ),
    legend_orientation="h"
)
fig1.show()


# fig2 = px.line(srrlandmeasured.reset_index(), x = 'Datetime', y = ['Ap_1', 'Ap_2', 'Ap_3', "Ap_4", "Ap_5"])
fig2 = px.line(measured.reset_index(), x = 'Datetime', y = ['Ap_1', 'Ap_2', 'Ap_3', "Ap_4", "Ap_5"], facet_row= "variable")
fig2.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Sensor location"
    ),
    legend_orientation="h"
)
fig2.show()